In [ ]:
#Karen

#Description:
#   Pulling in data from all sources.  These sources include:
#
#   FBI Crime Data API - https://crime-data-explorer.fr.cloud.gov/api
#   FBI Crime Data API:  lookups-controller : Endpoints pertaining to CDE Related Lookup Value data (LIST OF STATES & AGENCIES)
#   FBI Crime Data API:  victim-data-controller : Endpoints pertaining to NIBRS Victim Demographic data (NATIONAL & STATE - HOMICIDE ONLY)
#   FBI Crime Data API:  offense-data-controller : Endpoints pertaining to NIBRS Offender Demographic data (NATIONAL & STATE - HOMICIDE ONLY)
#   CENSUS DATA CSV:  Used https://factfinder.census.gov to build a table of the data.  Exported that data from the website into a csv file (2017 YEAR ONLY)  
#   MURDER RATES:  By State:  https://deathpenaltyinfo.org/facts-and-research/murder-rates/murder-rates-by-state
# 
#
#Modification History:
#   DD-MMM-YYY  Author          Description
#   10-07-2019  Stacey Smith    INITIAL CREATION
#   12-07-2019  Andrea Morgan   API loop build
#   17-07-2019  Stacey Smith    Add Agency and Economic/Census data


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import csv

from pprint import pprint

# Import DATA.GOV API key 
from api_keys import ses_dg_key

NATIONAL FBI CRIME DATA  - EXPLORATION AND CLEANUP - VICTIM DATA ONLY (NO OFFENDERS)


In [ ]:
#API Call to National-level victim-data-controller for HOMICIDE ONLY  
#FORMAT:  https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/{offense}/victim/national/{variable}?api_key=

n_age_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/national/age?api_key=" + ses_dg_key
n_ethnicity_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/national/ethnicity?api_key=" + ses_dg_key
n_sex_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/national/sex?api_key=" + ses_dg_key
n_race_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/national/race?api_key=" + ses_dg_key
n_relationship_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/national/relationship?api_key=" + ses_dg_key

#Create JSON Files for each Homicide Variable
n_age_response = requests.get(n_age_query_url).json()
n_ethnicity_response = requests.get(n_ethnicity_query_url).json()
n_sex_response = requests.get(n_sex_query_url).json()
n_race_response = requests.get(n_race_query_url).json()
n_relationship_response = requests.get(n_relationship_query_url).json()


In [ ]:
#NATIONAL VICTIMS OF HOMICIDE BY AGE RANGE

n_age_data = pd.DataFrame(n_age_response['results'])
n_age_data = n_age_data.sort_values(by='data_year', ascending=False)

n_age_data.to_csv("output_data/natl_age_data.csv")

n_age_data.head()

In [ ]:
#NATIONAL VICTIMS OF HOMICIDE  BY ETHNICITY

n_eth_data = pd.DataFrame(n_ethnicity_response['results'])
n_eth_data.sort_values(by='data_year', ascending=False)

n_eth_data.to_csv("output_data/natl_eth_data.csv")

n_eth_data.head()

In [ ]:
#NATIONAL VICTIMS OF HOMICIDE  BY SEX

n_sex_data = pd.DataFrame(n_sex_response['results'])
n_sex_data.sort_values(by='data_year', ascending=False)

n_sex_data.to_csv("output_data/natl_sex_data.csv")

n_sex_data.head()

In [ ]:
#NATIONAL VICTIMS OF HOMICIDE  BY RACE

n_race_data = pd.DataFrame(n_race_response['results'])
n_race_data.sort_values(by='data_year', ascending=False)

n_race_data.to_csv("output_data/natl_race_data.csv")

n_race_data.head()

In [ ]:
#NATIONAL VICTIMS OF HOMICIDE BY RELATIONSHIP CATEGORY (RELATIONSHIP OF VICTIM TO OFFENDER)

n_rel_data = pd.DataFrame(n_relationship_response['results'])
n_rel_data.sort_values(by='data_year', ascending=False)

n_rel_data.to_csv("output_data/natl_rel_data.csv")

n_rel_data.head()

STATE FBI CRIME DATA - EXPLORATION AND CLEANUP

In [3]:
#There are 3 pages of data for states. 
#This pulls the data for each page and combines it all into one usable dataframe: state_data

state_qry = "https://api.usa.gov/crime/fbi/sapi/api/states?api_key=" + ses_dg_key 
state_results_1 = requests.get(state_qry, params={'page':0}).json()
state_results_2 = requests.get(state_qry, params={'page':1}).json()
state_results_3 = requests.get(state_qry, params={'page':2}).json()

sr1_df = pd.DataFrame(state_results_1['results'])
sr2_df = pd.DataFrame(state_results_2['results'])
sr3_df = pd.DataFrame(state_results_3['results'])


state_data = pd.concat([sr1_df, sr2_df, sr3_df])
state_data.head()

,region_code,state_abbr,state_fips_code,state_id,state_name
0,4,AK,2.0,1,Alaska
1,3,AL,1.0,2,Alabama
2,3,AR,5.0,3,Arkansas
3,99,AS,60.0,4,American Samoa
4,4,AZ,4.0,5,Arizona


VICTIM DATA

In [17]:
#NUMBER OF VICTIMS OF HOMICIDE PER STATE BY AGE RANGE

#This is the same series created before
states = (state_data.iloc[:,1]).astype(str)

#Opened an empty list to initialize a dataframe
empty_list = []

v_ageDF = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
v_ageDF["State"] = ""
v_ageDF["Year"] = ""
v_ageDF["Unknown"] = ""
v_ageDF["0-9"] = ""
v_ageDF["10-19"] = ""
v_ageDF["20-29"] = ""
v_ageDF["30-39"] = ""
v_ageDF["40-49"] = ""
v_ageDF["50-59"] = ""
v_ageDF["60-69"] = ""
v_ageDF["70-79"] = ""
v_ageDF["80-89"] = ""
v_ageDF["90-99"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    v_age_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/states/"+ state +"/age?api_key=" + ses_dg_key
    v_age_response_state = requests.get(v_age_query_url).json()
    
    for i in v_age_response_state['results']:
        
        v_ageDF = v_ageDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Unknown': i['unknown'],
                          '0-9' : i['range_0_9'],
                          '10-19' : i['range_10_19'],
                          '20-29' : i['range_20_29'],
                          '30-39' : i['range_30_39'],
                          '40-49' : i['range_40_49'],
                          '50-59' : i['range_50_59'],
                          '60-69' : i['range_60_69'],
                          '70-79' : i['range_70_79'],
                          '80-89' : i['range_80_89'],
                          '90-99' : i['range_90_99']},
                          ignore_index = True)
    

v_ageDF.to_csv("output_data/state_v_age_data.csv")

v_ageDF.head()  

,State,Year,Unknown,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
0,AL,1991,28,11,54,131,105,49,16,20,12,7,1
1,AL,1992,12,10,33,67,70,43,16,15,7,5,0
2,AL,2006,0,0,0,2,0,0,0,0,0,0,0
3,AL,2008,0,0,0,0,0,1,1,0,0,0,0
4,AL,2009,0,0,0,1,1,0,0,0,0,0,0


In [6]:
#NUMBER OF VICTIMS OF HOMICIDE PER STATE BY SEX

states = (state_data.iloc[:,1]).astype(str)

#Opened an empty list to initialize a dataframe
empty_list = []

sexDF = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
sexDF["State"] = ""
sexDF["Year"] = ""
sexDF["Female Count"] = ""
sexDF["Male Count"] = ""
sexDF["Unknown"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    sex_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/states/"+ state +"/sex?api_key=" + ses_dg_key
    sex_response_state = requests.get(sex_query_url).json()
    
    for i in sex_response_state['results']:
        
        sexDF = sexDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Female Count': i['female_count'],
                          'Male Count' : i['male_count'],
                          'Unkown' : i['unknown']},
                          ignore_index = True)
    
    
sexDF.to_csv("output_data/state_v_sex_data.csv") 


sexDF.head()  

,State,Year,Female Count,Male Count,Unknown,Unkown
0,AL,1991,107,321,NaN,6.0
1,AL,1992,54,216,NaN,8.0
2,AL,2006,1,1,NaN,0.0
3,AL,2008,1,1,NaN,0.0
4,AL,2009,1,1,NaN,0.0


In [7]:
#NUMBER OF  VICTIMS OF HOMICIDE PER STATE BY RELATIONSHIP CATEGORY

#This is the same series created before
states = (state_data.iloc[:,1]).astype(str)

#Opened an empty list to initialize a dataframe
empty_list = []

v_relDF = pd.DataFrame(empty_list)


#Creating columns on list based on the returned data for one state
v_relDF["State"] = ""
v_relDF["Year"] = ""
v_relDF["Relationship Unknown"] = ""
v_relDF["Acquaintance"] = ""
v_relDF["Babysittee"] = ""
v_relDF["BF_GF"] = ""
v_relDF["Child"] = ""
v_relDF["Child_BF_GF"] = ""
v_relDF["Common Law Spouse"] = ""
v_relDF["Employee"] = ""
v_relDF["Employer"] = ""
v_relDF["Ex Spouse"] = ""
v_relDF["Friend"] = ""
v_relDF["Grandchild"] = ""
v_relDF["Grandparent"] = ""
v_relDF["Same Sex Relationship"] = ""
v_relDF["In Law"] = ""
v_relDF["Neighbor"] = ""
v_relDF["Offender"] = ""
v_relDF["Other Family Member"] = ""
v_relDF["Otherwise Known"] = ""
v_relDF["Parent"] = ""
v_relDF["Sibling"] = ""
v_relDF["Spouse"] = ""
v_relDF["Stepchild"] = ""
v_relDF["Stepparent"] = ""
v_relDF["Stepsibling"] = ""
v_relDF["Stranger"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    v_rel_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/states/"+ state +"/relationship?api_key=" + ses_dg_key
    v_rel_response_state = requests.get(v_rel_query_url).json()
    
    for i in v_rel_response_state['results']:
        
        v_relDF = v_relDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Relationship Unknown': i['relationship_unknown'],
                          'Acquaintance' : i['acquaintance'],
                          'Babysittee' : i['babysittee'],
                          'BF_GF' : i['boyfriend_girlfriend'],
                          'Child' : i['child'],
                          'Child_BF_GF' : i['child_boyfriend_girlfriend'],
                          'Common Law Spouse' : i['common_law_spouse'],
                          'Employee' : i['employee'],
                          'Employer' : i['employer'],
                          'Ex Spouse' : i['ex_spouse'],
                          'Friend' : i['friend'],
                          'Grandchild' : i['grandchild'],
                             'Grandparent' : i['grandparent'], 
                             'Same Sex Relationship' : i['homosexual_relationship'], 
                             'In Law' : i['in_law'], 
                             'Neighbor' : i['neighbor'], 
                             'Offender' : i['offender'], 
                             'Other Family Member' : i['other_family_member'], 
                             'Otherwise Known' : i['otherwise_known'], 
                             'Parent' : i['parent'], 
                             'Relationship Unkown' : i['relationship_unknown'], 
                             'Sibling' : i['sibling'],
                             'Spouse' : i['spouse'],
                             'Stepchild' : i['stepchild'],
                             'Stepparent' : i['stepparent'],
                             'Stepsibling' : i['stepsibling'],
                             'Stranger' : i['stranger']},
                          ignore_index = True)
    
    
v_relDF.to_csv("output_data/state_v_rel_data.csv") 

v_relDF.head()

,State,Year,Relationship Unknown,Acquaintance,Babysittee,BF_GF,Child,Child_BF_GF,Common Law Spouse,Employee,...,Other Family Member,Otherwise Known,Parent,Sibling,Spouse,Stepchild,Stepparent,Stepsibling,Stranger,Relationship Unkown
0,AL,1991,86,136,0,48,9,0,4,3,...,16,1,7,6,38,0,2,0,179,86.0
1,AL,1992,126,76,0,17,7,0,5,0,...,11,1,3,4,20,1,1,0,45,126.0
2,AL,2006,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0.0
3,AL,2008,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0.0
4,AL,2009,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0


OFFENDER DATA

In [19]:
#NUMBER HOMICIDE OFFENDERS PER STATE BY SEX

#Opened an empty list to initialize a dataframe
empty_list = []

o_ageDF = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
o_ageDF["State"] = ""
o_ageDF["Year"] = ""
o_ageDF["Unknown"] = ""
o_ageDF["0-9"] = ""
o_ageDF["10-19"] = ""
o_ageDF["20-29"] = ""
o_ageDF["30-39"] = ""
o_ageDF["40-49"] = ""
o_ageDF["50-59"] = ""
o_ageDF["60-69"] = ""
o_ageDF["70-79"] = ""
o_ageDF["80-89"] = ""
o_ageDF["90-99"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    o_age_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/offender/states/"+ state +"/age?api_key=" + ses_dg_key
    o_age_response_state = requests.get(o_age_query_url).json()
    
    for i in o_age_response_state['results']:
        
        o_ageDF = o_ageDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Unknown': i['unknown'],
                          '0-9' : i['range_0_9'],
                          '10-19' : i['range_10_19'],
                          '20-29' : i['range_20_29'],
                          '30-39' : i['range_30_39'],
                          '40-49' : i['range_40_49'],
                          '50-59' : i['range_50_59'],
                          '60-69' : i['range_60_69'],
                          '70-79' : i['range_70_79'],
                          '80-89' : i['range_80_89'],
                          '90-99' : i['range_90_99']},
                          ignore_index = True)
    
o_ageDF.to_csv("output_data/state_o_age_data.csv") 

o_ageDF.head() 

,State,Year,Unknown,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
0,AL,1991,353,0,51,82,72,30,11,5,1,2,0
1,AL,1992,102,0,46,94,52,36,13,6,4,1,0
2,AL,2006,0,0,3,2,0,0,0,0,0,0,0
3,AL,2008,0,0,2,0,1,0,0,0,0,0,0
4,AL,2009,0,0,0,1,0,2,0,0,0,0,0


In [20]:
#NUMBER OF HOMICIDE OFFENDERS PER STATE BY SEX

#Opened an empty list to initialize a dataframe
empty_list = []

o_sexDF = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
o_sexDF["State"] = ""
o_sexDF["Year"] = ""
o_sexDF["Female Count"] = ""
o_sexDF["Male Count"] = ""
o_sexDF["Unknown"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    o_sex_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/offender/states/"+ state +"/sex?api_key=" + ses_dg_key
    o_sex_response_state = requests.get(o_sex_query_url).json()
    
    for i in o_sex_response_state['results']:
        
        o_sexDF = o_sexDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Female Count': i['female_count'],
                          'Male Count' : i['male_count'],
                          'Unkown' : i['unknown']},
                          ignore_index = True)
    
    
o_sexDF.to_csv("output_data/state_o_sex_data.csv") 

o_sexDF.head()

,State,Year,Female Count,Male Count,Unknown,Unkown
0,AL,1991,44,237,NaN,326.0
1,AL,1992,48,240,NaN,66.0
2,AL,2006,0,5,NaN,0.0
3,AL,2008,0,3,NaN,0.0
4,AL,2009,1,2,NaN,0.0


AGENCY FBI CRIME DATA - EXPLORATION AND CLEANUP

In [21]:
#AGENCY DATA BY STATE - RAW DATA
#Opened an empty list to initialize a dataframe
empty_list = []

agencies = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
agencies["State"] = ""
agencies["Origin"] = ""
agencies["Agency Name"] = ""
agencies["Agency Type Name"] = ""
agencies["State Abbr"] = ""
agencies["Division Name"] = ""
agencies["Region Name"] = ""
agencies["Region Desc"] = ""
agencies["County Name"] = ""
agencies["NIBRS"] = ""
agencies["Latitude"] = ""          
agencies["Longitude"] = ""
agencies["NIBRS Start Date"] = ""

#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    agency_query_url = "https://api.usa.gov/crime/fbi/sapi/api/agencies/byStateAbbr/"+ state +"?api_key=" + ses_dg_key
    agency_response_state = requests.get(agency_query_url).json()
    
    for i in agency_response_state['results']:
        
        agencies = agencies.append({'State': state, 
                          'Origin': i['ori'], 
                          'Agency Name': i['agency_name'],
                          'Agency Type Name' : i['agency_type_name'],
                          'State Abbr' : i['state_abbr'],
                          'Division Name' : i['division_name'],
                          'Region Name' : i['region_name'],
                          'Region Desc' : i['region_desc'],
                          'County Name' : i['county_name'],
                          'NIBRS' : i['nibrs'],
                          'Latitude' : i['latitude'],
                          'Longitude' : i['longitude'],
                          'NIBRS Start Date' : i['nibrs_start_date']},
                          ignore_index = True)
    

agencies.to_csv("output_data/raw_agencies.csv") 

agencies.head()

,State,Origin,Agency Name,Agency Type Name,State Abbr,Division Name,Region Name,Region Desc,County Name,NIBRS,Latitude,Longitude,NIBRS Start Date
0,AK,AK0010100,Anchorage Police Department,City,AK,Pacific,West,Region IV,ANCHORAGE,False,61.1743,-149.284,None
1,AK,AK0010200,Fairbanks Police Department,City,AK,Pacific,West,Region IV,FAIRBANKS NORTH STAR,False,64.8394,-147.719,None
2,AK,AK0010300,Juneau Police Department,City,AK,Pacific,West,Region IV,JUNEAU,False,58.3566,-134.507,None
3,AK,AK0010400,Ketchikan Police Department,City,AK,Pacific,West,Region IV,KETCHIKAN GATEWAY,False,55.4499,-131.107,None
4,AK,AK0010500,Kodiak Police Department,City,AK,Pacific,West,Region IV,KODIAK ISLAND,False,57.8049,-152.373,None


In [22]:
#AGENCY DATA CLEANUP
#NEED TO FIND THE NUMBERS OF AGENCIES REPORTING DATA TO THE FBI 

#Gives me a total count of all agencies, both True and False for NIBRS
NIBRS_data = agencies.groupby(['State']).count()

#Gives me a series that contains just the counts of True and False NIBRS by State
NIBRS_data_s = agencies.groupby(['State', 'NIBRS']).size()

#Put the series into a dataframe
NIBRS_df = pd.DataFrame(NIBRS_data_s)
NIBRS_df.reset_index(level=['State', 'NIBRS'], inplace=True)

#Merge the dataframes
all_agency = pd.merge(NIBRS_data, NIBRS_df, on=['State'])

#Cleanup columns I don't want
del all_agency['Origin']
del all_agency['Agency Name']
del all_agency['Agency Type Name']
del all_agency['State Abbr']
del all_agency['Division Name']
del all_agency['Region Name']
del all_agency['Region Desc']
del all_agency['County Name']
del all_agency['Latitude']
del all_agency['Longitude']
del all_agency['NIBRS Start Date']

#Create final dataframe
t_all_agency = all_agency.loc[all_agency['NIBRS_y']==True, :]

all_agencies = t_all_agency.rename(columns={
    "NIBRS_x": "Total Agencies",
    0: "Active Agencies"
    
})

#Get rid of one more column
del all_agencies['NIBRS_y']

#Add a new column that calculates the percentage of agencies reporting data to the FBI
all_agencies['Pct Active'] = (all_agencies['Active Agencies']/all_agencies['Total Agencies'])*100

all_agencies.to_csv("output_data/final_agency_data.csv")

all_agencies.head()

,State,Total Agencies,Active Agencies,Pct Active
2,AL,423,1,0.236407
4,AR,310,294,94.838710
6,AZ,124,5,4.032258
9,CO,244,220,90.163934
11,CT,107,94,87.850467


CENSUS DATA - EXPLORATION AND CLEANUP

In [24]:
#Open the Census Economic data CSV file
path_econ = "Resources/ACS_17_5YR_DP03_with_ann.csv"
econ_df = pd.read_csv(path_econ, encoding="ISO-8859-1")


#CLEAN AND ORGANIZE THE DATA
# For the purposes of this analysis, I only want to use a small portion of the data I downloaded from the census website. 
 
# Delete unwanted data 
del econ_df['Id2']
del econ_df['Estimate; EMPLOYMENT STATUS - Civilian labor force']
del econ_df['Percent; EMPLOYMENT STATUS - Civilian labor force']
del econ_df['Percent; EMPLOYMENT STATUS - Civilian labor force - Unemployment Rate']
del econ_df['Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years - Related children of the householder under 18 years'] 
del econ_df['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years - Related children of the householder under 18 years']
del econ_df['Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years - Related children of the householder under 18 years - Related children of the householder 5 to 17 years']
del econ_df['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years - Related children of the householder under 18 years - Related children of the householder 5 to 17 years']
del econ_df['Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 18 to 64 years']
del econ_df['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 18 to 64 years']
del econ_df['Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over']
del econ_df['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over - 65 years and over']
del econ_df['Estimate; EMPLOYMENT STATUS - Females 16 years and over - In labor force'],
del econ_df['Percent; EMPLOYMENT STATUS - Females 16 years and over - In labor force'],
del econ_df['Estimate; EMPLOYMENT STATUS - Females 16 years and over - In labor force - Civilian labor force'],
del econ_df['Percent; EMPLOYMENT STATUS - Females 16 years and over - In labor force - Civilian labor force'],
del econ_df['Estimate; EMPLOYMENT STATUS - Females 16 years and over - In labor force - Civilian labor force - Employed'],
del econ_df['Percent; EMPLOYMENT STATUS - Females 16 years and over - In labor force - Civilian labor force - Employed'],
del econ_df['Estimate; COMMUTING TO WORK - Workers 16 years and over'],
del econ_df['Percent; COMMUTING TO WORK - Workers 16 years and over'],
del econ_df['Estimate; COMMUTING TO WORK - Workers 16 years and over - Car, truck, or van -- drove alone'],
del econ_df['Percent; COMMUTING TO WORK - Workers 16 years and over - Car, truck, or van -- drove alone'],
del econ_df['Estimate; COMMUTING TO WORK - Workers 16 years and over - Car, truck, or van -- carpooled'],
del econ_df['Percent; COMMUTING TO WORK - Workers 16 years and over - Car, truck, or van -- carpooled'],
del econ_df['Estimate; COMMUTING TO WORK - Workers 16 years and over - Public transportation (excluding taxicab)'],
del econ_df['Percent; COMMUTING TO WORK - Workers 16 years and over - Public transportation (excluding taxicab)'],
del econ_df['Estimate; COMMUTING TO WORK - Workers 16 years and over - Walked'],
del econ_df['Percent; COMMUTING TO WORK - Workers 16 years and over - Walked'],
del econ_df['Estimate; COMMUTING TO WORK - Workers 16 years and over - Other means'],
del econ_df['Percent; COMMUTING TO WORK - Workers 16 years and over - Other means'],
del econ_df['Estimate; COMMUTING TO WORK - Workers 16 years and over - Worked at home'],
del econ_df['Percent; COMMUTING TO WORK - Workers 16 years and over - Worked at home'],
del econ_df['Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)'],
del econ_df['Estimate; OCCUPATION - Civilian employed population 16 years and over'],
del econ_df['Percent; OCCUPATION - Civilian employed population 16 years and over'],
del econ_df['Estimate; OCCUPATION - Civilian employed population 16 years and over - Management, business, science, and arts occupations'],
del econ_df['Percent; OCCUPATION - Civilian employed population 16 years and over - Management, business, science, and arts occupations'],
del econ_df['Estimate; OCCUPATION - Civilian employed population 16 years and over - Service occupations'],
del econ_df['Percent; OCCUPATION - Civilian employed population 16 years and over - Service occupations'],
del econ_df['Estimate; OCCUPATION - Civilian employed population 16 years and over - Sales and office occupations'],
del econ_df['Percent; OCCUPATION - Civilian employed population 16 years and over - Sales and office occupations'],
del econ_df['Estimate; OCCUPATION - Civilian employed population 16 years and over - Natural resources, construction, and maintenance occupations'],
del econ_df['Percent; OCCUPATION - Civilian employed population 16 years and over - Natural resources, construction, and maintenance occupations'],
del econ_df['Estimate; OCCUPATION - Civilian employed population 16 years and over - Production, transportation, and material moving occupations'],
del econ_df['Percent; OCCUPATION - Civilian employed population 16 years and over - Production, transportation, and material moving occupations'],
del econ_df['Estimate; CLASS OF WORKER - Civilian employed population 16 years and over'],
del econ_df['Percent; CLASS OF WORKER - Civilian employed population 16 years and over'],
del econ_df['Estimate; CLASS OF WORKER - Civilian employed population 16 years and over - Private wage and salary workers'],
del econ_df['Percent; CLASS OF WORKER - Civilian employed population 16 years and over - Private wage and salary workers'],
del econ_df['Estimate; CLASS OF WORKER - Civilian employed population 16 years and over - Government workers'],
del econ_df['Percent; CLASS OF WORKER - Civilian employed population 16 years and over - Government workers'],
del econ_df['Estimate; CLASS OF WORKER - Civilian employed population 16 years and over - Self-employed in own not incorporated business workers'],
del econ_df['Percent; CLASS OF WORKER - Civilian employed population 16 years and over - Self-employed in own not incorporated business workers'],
del econ_df['Estimate; CLASS OF WORKER - Civilian employed population 16 years and over - Unpaid family workers'],
del econ_df['Percent; CLASS OF WORKER - Civilian employed population 16 years and over - Unpaid family workers'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Less than $10,000'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Less than $10,000'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $10,000 to $14,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $10,000 to $14,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $15,000 to $24,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $15,000 to $24,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $25,000 to $34,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $25,000 to $34,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $35,000 to $49,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $35,000 to $49,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $50,000 to $74,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $50,000 to $74,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $75,000 to $99,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $75,000 to $99,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $100,000 to $149,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $100,000 to $149,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $150,000 to $199,999'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $150,000 to $199,999'],
del econ_df['Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $200,000 or more'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $200,000 or more'],
del econ_df['Estimate; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population'],
del econ_df['Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population'],
del econ_df['Estimate; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage'],
del econ_df['Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage'],
del econ_df['Estimate; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With private health insurance'],
del econ_df['Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With private health insurance'],
del econ_df['Estimate; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With public coverage'],
del econ_df['Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - With health insurance coverage - With public coverage'],
del econ_df['Estimate; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage'],
del econ_df['Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage'],
del econ_df['Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people'],
del econ_df['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people'],
del econ_df['Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years'],
del econ_df['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - Under 18 years'],
del econ_df['Estimate; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars)'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for male full-time, year-round workers (dollars)'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Median household income (dollars)'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for female full-time, year-round workers (dollars)'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Mean household income (dollars)'],
del econ_df['Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)']


# Original column names are ridiculous.  Shortening them to something more manageable.
# Also need to make sure the 'Geography' column is renamed to 'state_name' so I can merge with my state data later on.

# Rename columns and create a new dataframe from the result
o_econ_df = econ_df.rename(index=str, columns={
    
    'Geography': "state_name", 
    'Estimate; EMPLOYMENT STATUS - Population 16 years and over': "Emp_pop_all",
    'Percent; EMPLOYMENT STATUS - Population 16 years and over': "Pct_Emp_pop_all",
    'Estimate; EMPLOYMENT STATUS - Population 16 years and over - Not in labor force': "UnEmp_pop_all",
    'Percent; EMPLOYMENT STATUS - Population 16 years and over - Not in labor force': "Pct_UnEmp_pop_all",
    'Estimate; EMPLOYMENT STATUS - Females 16 years and over': "Emp_pop_Female",
    'Percent; EMPLOYMENT STATUS - Females 16 years and over': "Pct_Emp_pop_Female",
    'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people - 18 years and over': "Pct_Pov_pop_all18",
    'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Median household income (dollars)': "Total_median_income",
    'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Mean household income (dollars)': "Total_mean_income",
    'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)': "Per_capita_income",
    'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars)': "Median_earnings",
    'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for male full-time, year-round workers (dollars)': "Median_male",
    'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Median earnings for female full-time, year-round workers (dollars)': "Median_female"

   
})

# Merge the state data and my econ data on the state_name.  This allows me to bring the state_abbr column into the df, which I will need later on. 
o_econ_df_s = pd.merge(o_econ_df, state_data, on="state_name")

o_econ_df_s.to_csv("output_data/census_data.csv")

o_econ_df_s.head()

,state_name,Emp_pop_all,Pct_Emp_pop_all,UnEmp_pop_all,Pct_UnEmp_pop_all,Emp_pop_Female,Pct_Emp_pop_Female,Total_median_income,Total_mean_income,Per_capita_income,Median_earnings,Median_male,Median_female,Pct_Pov_pop_all18,region_code,state_abbr,state_fips_code,state_id
0,Alabama,3876136,3876136,1642927,42.4,2022516,2022516,46472,64476,25746,29288,46602,35060,15.6,3,AL,1.0,2
1,Alaska,571453,571453,170831,29.9,271063,271063,76114,94469,35065,37369,58955,47160,8.9,4,AK,2.0,1
2,Arizona,5371341,5371341,2174225,40.5,2718818,2718818,53510,73735,27964,30811,46294,38810,14.7,4,AZ,4.0,5
3,Arkansas,2351061,2351061,984959,41.9,1209967,1209967,43813,61330,24426,27923,41576,32813,15.8,3,AR,5.0,3
4,California,30910058,30910058,11297281,36.5,15665864,15665864,67169,96104,33128,32738,52054,45568,13.4,4,CA,6.0,6


In [30]:
#MURDER RATES AND DEATH PENALTY BY STATE
#I only want data for 2017

#IMPORT MURDER RATE & DEATH PENALTY BY STATE (DEATH PENALTY.ORG) 
ses_mr_path = "Resources/Murder_Rates_By_State_data.csv"
ses_mr_df = pd.read_csv(ses_mr_path, encoding="ISO-8859-1")
ses_mr_df = ses_mr_df.rename(columns={"State": "state_name"})
ses_mr_2017 = ses_mr_df.loc[ses_mr_df['Year']=='1/1/2017', :]

ses_mr_2017.to_csv("output_data/Murder_Rates_and_Death_Penalty_by_State_2017.csv")

ses_mr_2017.head()

,Death Penalty Status,Murder Rate,Number of Records,Region,state_name,Year
1010,Region,5.7,1,Midwest,x-Midwest,1/1/2017
1041,Region,3.5,1,Northeast,x-Northeast,1/1/2017
1072,Region,6.4,1,South,x-South,1/1/2017
1103,Region,5.3,1,United States,x-United States,1/1/2017
1134,Region,4.5,1,West,x-West,1/1/2017
